__Random Monkey Stock Picker__

In [6]:
# load a list of all S&P 500 companies
import pandas as pd

df = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = df[0]
df = df[['Symbol', 'Security', 'GICS Sector']]
df['Ticker'] = df['Symbol']
df['Company'] = df['Security']
df = df[['Ticker', 'Company', 'GICS Sector']]
df.head()

,Ticker,Company,GICS Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott,Health Care
3,ABBV,AbbVie,Health Care
4,ACN,Accenture,Information Technology


Generate a random stock picker

In [7]:
# implementing random stock picker to build portfolio of 10 stocks
import random

tickers = df['Ticker'].tolist()
random_tickers = random.sample(tickers, 10)
Portfolio = df[df['Ticker'].isin(random_tickers)]

In [8]:
Portfolio

,Ticker,Company,GICS Sector
7,ADBE,Adobe Inc.,Information Technology
53,ADSK,Autodesk,Information Technology
118,KO,Coca-Cola Company (The),Consumer Staples
158,DOW,Dow Inc.,Materials
248,IEX,IDEX Corporation,Industrials
256,ICE,Intercontinental Exchange,Financials
301,MKTX,MarketAxess,Financials
360,OGN,Organon & Co.,Health Care
382,PFG,Principal Financial Group,Financials
469,VRSN,Verisign,Information Technology


As of April 2nd this portfolio has a YTD return of 3.83%, compared to the S&P 500 return of 7.46%
My personal YTD return is 7.56% and thus beats both :)